In [1]:
%cd ../envs

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import gym

In [3]:
from envs import make_rl_envs, make_env

ModuleNotFoundError: No module named 'envs'

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(10, 10))
display.start()

In [ ]:
env_name = "Mazelab-v2"
# env_name = "MazelabRandom-v1"
# env_name = "CartPole-v0"
# env_name = "cartpole-swingup"

env = make_env(env_name)

In [ ]:
s = env.reset()
print(s.shape)
plt.imshow(s.transpose(1, 2, 0))


In [ ]:
# a = np.array(np.random.rand(1)*100)
# a = np.clip(a, env.action_space.low, env.action_space.high)
# s, _, done, _ = env.step(env.action_space.sample())
s, _, done, _ = env.step(3)
plt.imshow(s.transpose(1, 2, 0))

if done:
    s = env.reset()
    plt.imshow(s.transpose(1, 2, 0))
    print(done)


In [ ]:
envs = make_rl_envs(
    env_name,
    0,
    2,
    None,
    frame_stack=1
)

In [ ]:
s = envs.reset()
plt.imshow(s[1].numpy().transpose(1, 2, 0))

In [ ]:
s.shape

In [ ]:
import torch
import os
import time
from collections import deque

import gym
import numpy as np

In [ ]:
from a2c_ppo_acktr import algo, utils
from a2c_ppo_acktr.algo import gail
from a2c_ppo_acktr.arguments import get_args
from a2c_ppo_acktr.envs import make_vec_envs
from a2c_ppo_acktr.model import Policy
from a2c_ppo_acktr.storage import RolloutStorage
from evaluation import evaluate

In [ ]:
import time
seed = int(time.time()*1000)
print(seed)
cuda = True
log_dir = "log"
# env_name = "Mazelab-v1"
env_name = "Mazelab-v2"
# env_name = "Mazelab-v3"
# env_name = "Mazelab-v4"
# env_name = "Mazelab-v5"

num_envs = 32
recurrent_policy = False
algo_type = "ppo"

value_loss_coef = 0.5
lr = 2.5e-4
entropy_coef = 0.01
eps = 1e-5
alpha = 0.99
max_grad_norm = 0.5
clip_param = 0.2
ppo_epoch = 4
num_mini_batch = 32
gamma = 0.99
num_steps = 128
num_env_steps = 1e7
use_linear_lr_decay = True

use_gae = False
gae_lambda = 0.95
use_proper_time_limits = False
save_interval = 100
save_dir = "save"
log_interval = 5
eval_interval = 10



torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if cuda and torch.cuda.is_available():
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

log_dir = os.path.expanduser(log_dir)
eval_log_dir = log_dir + "_eval"
utils.cleanup_log_dir(log_dir)
utils.cleanup_log_dir(eval_log_dir)

torch.set_num_threads(1)
device = torch.device("cuda:0" if cuda else "cpu")


envs = make_rl_envs(
    env_name,
    seed,
    num_envs,
    device,
    frame_stack=1
)


actor_critic = Policy(
    envs.observation_space.shape,
    envs.action_space,
    base_kwargs={'recurrent': recurrent_policy})
actor_critic.to(device)



if algo_type == 'a2c':
    agent = algo.A2C_ACKTR(
        actor_critic,
        value_loss_coef,
        entropy_coef,
        lr=lr,
        eps=eps,
        alpha=alpha,
        max_grad_norm=max_grad_norm)
elif algo_type == 'ppo':
    agent = algo.PPO(
        actor_critic,
        clip_param,
        ppo_epoch,
        num_mini_batch,
        value_loss_coef,
        entropy_coef,
        lr=lr,
        eps=eps,
        max_grad_norm=max_grad_norm)
elif algo_type == 'acktr':
    agent = algo.A2C_ACKTR(
        actor_critic, value_loss_coef, entropy_coef, acktr=True)


rollouts = RolloutStorage(num_steps, num_envs,
                          envs.observation_space.shape, envs.action_space,
                          actor_critic.recurrent_hidden_state_size)

obs = envs.reset()
rollouts.obs[0].copy_(obs)
rollouts.to(device)

episode_rewards = deque(maxlen=30)

In [ ]:
def evaluate(actor_critic, env_name, seed, num_processes, eval_log_dir,
             device):
    eval_envs = make_rl_envs(env_name, seed + num_processes, num_processes,
                              device, frame_stack=1)
    eval_episode_rewards = deque(maxlen=30)

    obs = eval_envs.reset()
    eval_recurrent_hidden_states = torch.zeros(
        num_processes, actor_critic.recurrent_hidden_state_size, device=device)
    eval_masks = torch.zeros(num_processes, 1, device=device)

    while len(eval_episode_rewards) < 30:
        with torch.no_grad():
            _, action, _, eval_recurrent_hidden_states = actor_critic.act(
                obs,
                eval_recurrent_hidden_states,
                eval_masks,
                deterministic=True)

        # Obser reward and next obs
        obs, _, done, infos = eval_envs.step(action)

        eval_masks = torch.tensor(
            [[0.0] if done_ else [1.0] for done_ in done],
            dtype=torch.float32,
            device=device)

        for info in infos:
            if 'episode' in info.keys():
                eval_episode_rewards.append(info['episode']['r'])

    eval_envs.close()
    return eval_episode_rewards


In [ ]:
plt.imshow(obs[0].cpu().numpy().transpose(1, 2, 0)[:,:,:3])

In [ ]:
start = time.time()
num_updates = int(num_env_steps) // num_steps // num_envs
for j in range(num_updates):

    if use_linear_lr_decay:
        # decrease learning rate linearly
        utils.update_linear_schedule(
            agent.optimizer, j, num_updates,
            agent.optimizer.lr if algo_type == "acktr" else lr)

    for step in range(num_steps):
        # Sample actions
        with torch.no_grad():
            value, action, action_log_prob, recurrent_hidden_states = actor_critic.act(
                rollouts.obs[step], rollouts.recurrent_hidden_states[step],
                rollouts.masks[step])

        # Obser reward and next obs
        obs, reward, done, infos = envs.step(action)

        for info in infos:
            if 'episode' in info.keys():
                episode_rewards.append(info['episode']['r'])

        # If done then clean the history of observations.
        masks = torch.FloatTensor(
            [[0.0] if done_ else [1.0] for done_ in done])
        bad_masks = torch.FloatTensor(
            [[0.0] if 'bad_transition' in info.keys() else [1.0]
             for info in infos])
        rollouts.insert(obs, recurrent_hidden_states, action,
                        action_log_prob, value, reward, masks, bad_masks)

    with torch.no_grad():
        next_value = actor_critic.get_value(
            rollouts.obs[-1], rollouts.recurrent_hidden_states[-1],
            rollouts.masks[-1]).detach()



    rollouts.compute_returns(next_value, use_gae, gamma,
                             gae_lambda, use_proper_time_limits)

    value_loss, action_loss, dist_entropy = agent.update(rollouts)

    rollouts.after_update()




    # save for every interval-th episode or for the last epoch
    if (j % save_interval == 0
            or j == num_updates - 1) and save_dir != "":
        save_path = os.path.join(save_dir, algo_type)
        try:
            os.makedirs(save_path)
        except OSError:
            pass

        torch.save([
            actor_critic,
            getattr(utils.get_vec_normalize(envs), 'ob_rms', None)
        ], os.path.join(save_path, env_name + ".pt"))

    if j % log_interval == 0 and len(episode_rewards) > 1:
        total_num_steps = (j + 1) * num_envs * num_steps
        end = time.time()
        print(
            """Updates {}, num timesteps {}, FPS {}
            Last {} training episodes: mean/median reward {:.1f}/{:.1f}, min/max reward {:.1f}/{:.1f}
            entropy: {:.2f} value loss: {:.3f} action loss {:.3f}
            """.format(
                j, total_num_steps,
                int(total_num_steps / (end - start)),
                len(episode_rewards), np.mean(episode_rewards),
                np.median(episode_rewards), np.min(episode_rewards),
                np.max(episode_rewards),
                dist_entropy, value_loss, action_loss)
        )

    if (eval_interval is not None and len(episode_rewards) > 1
            and j % eval_interval == 0):
        eval_episode_rewards = evaluate(
            actor_critic, env_name, seed,
            num_envs, eval_log_dir, device)
        print(
            "{} eval episodes: mean/median reward {:.1f}/{:.1f}, min/max reward {:.1f}/{:.1f}\n"
            .format(
                len(eval_episode_rewards), np.mean(eval_episode_rewards),
                np.median(eval_episode_rewards), np.min(eval_episode_rewards),
                np.max(eval_episode_rewards))
        )

In [ ]:
rollouts.obs.shape

In [ ]:
obs_seq = rollouts.obs[:, 0, 2, :, :].cpu().numpy()

In [ ]:
plt.imshow(obs_seq[100:109,:,:].reshape(-1, 64))